In [9]:
'''
批训练：把数据分为一小批一小批进行训练
Dataloader就是用来包装使用的数据，
比如说该程序中把数据5个5个的打包，
每一次抛出一组数据进行操作。
'''
import torch
import torch.utils.data as Data
import pandas as pd
from sklearn import preprocessing
import numpy as np
torch.manual_seed(1)
BATCH_SIZE = 5

df = pd.DataFrame(data=preprocessing.StandardScaler().fit_transform(np.random.randint(0, 10, size=(20, 10))))
#    y = pd.Series(np.random.randint(0, 2, 200))
def get_tensor_from_pd(dataframe_series) -> torch.Tensor:
    return torch.tensor(data=dataframe_series.values)
x = get_tensor_from_pd(df).float()#, get_tensor_from_pd(y).float()
# x = np.random.randint(0, 10, size=(200, 5))
y = torch.linspace(11,21,20)
 
torch_dataset = Data.TensorDataset(x,y) #把数据放在数据库中
loader = Data.DataLoader(
    # 从dataset数据库中每次抽出batch_size个数据
    dataset=torch_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,#将数据打乱
    num_workers=2, #使用两个线程
)
def show_batch():
    for epoch in range(3): #对全部数据进行3次训练
        for step,(batch_x,batch_y) in enumerate(loader): # 每一次挑选出来的size个数据
 
            # training
 
            # 打印出来，观察数据
            print('Epoch:',epoch,'|Step:',step,'|batch x:',
                  batch_x.numpy(),'|batch y:',batch_y.numpy())


In [28]:
# show_batch()

In [41]:
from chusai import *
from scipy.io import savemat
PathSet = {0:"./TestData", 1:"./CompetitionData1", 2:"./CompetitionData2", 3:"./CompetitionData3", 4:"./CompetitionData4"}
PrefixSet = {0:"Test" , 1:"Round1", 2:"Round2", 3:"Round3", 4:"Round4"}

Ridx = 0 # 设置比赛轮次索引，指明数据存放目录。0:Test; 1: 1st round; 2: 2nd round ...
PathRaw = "./chusai_data/" + PathSet[Ridx]
PathOut = "./outputs/" + PathSet[Ridx]
PathTrain = './train_data/' 
Prefix = PrefixSet[Ridx]

## 1查找文件
names= FindFiles(PathRaw) # 查找文件夹中包含的所有比赛/测试数据文件，非本轮次数据请不要放在目标文件夹中

dirs = os.listdir(PathRaw)
names = []  # 文件编号
files = []
for f in sorted(dirs):
    if f.endswith('.txt'):
        files.append(f)
for f in sorted(files):
    if f.find('CfgData')!=-1 and f.endswith('.txt'):
        names.append(f.split('CfgData')[-1].split('.txt')[0])

## 2创建对象并处理
Rst = []
Gt  = []
# for na in names: #
for na in [names[0]]:#
    # 读取配置及CSI数据
    Cfg = CfgFormat(PathRaw + '/' + Prefix + 'CfgData' + na + '.txt')
    csi = np.genfromtxt(PathRaw + '/' + Prefix + 'InputData' + na + '.txt', dtype = float)
    CSI = csi[:,0::2] + 1j* csi[:,1::2]

Now reading file TestCfgData001.txt ...

Now reading file TestCfgData002.txt ...

Now reading file TestCfgData003.txt ...

Now reading file TestCfgData004.txt ...

Now reading file TestCfgData005.txt ...

Now reading file TestCfgData006.txt ...

Now reading file TestCfgData007.txt ...

Now reading file TestCfgData008.txt ...

Now reading file TestCfgData009.txt ...



In [12]:
Cfg

{'Nsamp': 68,
 'Np': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 3, 3, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1]),
 'Ntx': 1,
 'Nrx': 3,
 'Nsc': 30,
 'Nt': array([1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500,
        1501, 1501, 1501, 1501, 1501, 1501, 1501, 1501, 1501, 1501, 1501,
        1501, 1501, 1501, 1501, 1501, 1501, 1501, 1501, 1501, 1501, 6000,
        6000, 6000, 6000, 6000, 4800, 4800, 1500, 1500, 4800, 4800, 4800,
        4800, 4800, 4800, 4800, 1500, 1500, 1500, 1500, 1500, 1500, 1500,
        1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500,
        1500, 1500]),
 'Tdur': array([30.059486, 30.299162, 30.299324, 30.099274, 30.059309, 30.099408,
        30.399524, 30.059742, 30.03952 , 30.059552, 29.99948 , 30.159336,
        30.219226, 30.339485, 30.479488, 30.259349, 30.879334, 30.499545,
      

In [13]:
CSI.shape

(154221, 90)

In [17]:
with open(PathRaw + '/' + Prefix + 'GroundTruthData' + na + '.txt', 'r') as f:
    gt = [np.fromstring(arr.strip(), dtype=float, sep = ' ') for arr in f.readlines()]
gt

[array([22.440194]),
 array([22.428071]),
 array([21.130579]),
 array([20.390532]),
 array([20.399487]),
 array([20.586328]),
 array([18.350859]),
 array([21.685914]),
 array([19.647731]),
 array([20.759867]),
 array([19.461718]),
 array([16.309124]),
 array([17.981956]),
 array([19.650385]),
 array([20.190844]),
 array([20.013712]),
 array([19.466318]),
 array([20.387775]),
 array([20.204478]),
 array([19.825724]),
 array([20.020472]),
 array([18.896096]),
 array([24.639865]),
 array([19.65238]),
 array([17.416404]),
 array([19.457776]),
 array([19.456463]),
 array([19.83242]),
 array([16.497238]),
 array([25.572213]),
 array([20.388464]),
 array([17.422281]),
 array([14.939751]),
 array([19.569505]),
 array([36.626195]),
 array([18.91996]),
 array([22.809468]),
 array([10.649476, 23.225623]),
 array([ 8.147241, 21.925367]),
 array([11.120982, 15.02144 , 27.102235]),
 array([11.116103, 14.650046, 21.718914]),
 array([14.446246]),
 array([14.350254]),
 array([15.462023]),
 array([15.83

In [30]:
from scipy.signal import savgol_filter

In [44]:
Nt = [0] + list(accumulate(Cfg['Nt']))
# 记录Np=1、Nt<1600的CSI样本有多少个
CSI_Np1_Nt1600_cnt = 0
for ii in range(Cfg['Nsamp']):
    # CSI_s.append(CsiFormatConvrt(CSI[Nt[ii]:Nt[ii+1],:], Cfg['Nrx'],
    #                                     Cfg['Ntx'], Cfg['Nsc'], Cfg['Nt'][ii]))
    if Cfg['Np'][ii] == 1 and Cfg['Nt'][ii] < 1600:
        CSI_Np1_Nt1600_cnt += 1


# 生成训练数据集
CSI_train = np.zeros((CSI_Np1_Nt1600_cnt*CSI.shape[1], 1500), dtype = complex) # CSI序列
GT_train = np.zeros((CSI_Np1_Nt1600_cnt*CSI.shape[1], 1), dtype = float) # 呼吸速率真实值
CSI_Np1_Nt1600_cnt = 0
for ii in range(Cfg['Nsamp']):
    if Cfg['Np'][ii] == 1 and Cfg['Nt'][ii] < 1600:
        CSI_Np1_Nt1600_cnt += 1
        CSI_Nsamp = np.transpose(CSI[Nt[ii]:Nt[ii+1],:]) #90,1500
        CSI_train[(CSI_Np1_Nt1600_cnt-1)*CSI.shape[1]:CSI_Np1_Nt1600_cnt*CSI.shape[1],:] = CSI_Nsamp[:,0:1500]
        GT_train[(CSI_Np1_Nt1600_cnt-1)*CSI.shape[1]:CSI_Np1_Nt1600_cnt*CSI.shape[1],0] = gt[ii]

# CSI序列实虚部分开,savgol_filter滤波，归一化,最后把实虚部分作为第三个维度合并
CSI_train_real = savgol_filter(np.real(CSI_train), 8, 7, axis=1)
CSI_train_imag = savgol_filter(np.imag(CSI_train), 8, 7, axis=1)
CSI_train_real = preprocessing.StandardScaler().fit_transform(CSI_train_real)
CSI_train_imag = preprocessing.StandardScaler().fit_transform(CSI_train_imag)
CSI_train_conca = np.concatenate((CSI_train_real[:,:,np.newaxis], CSI_train_imag[:,:,np.newaxis]), axis=2)

# for i in range(CSI_train_conca.shape[0]):
#     print(np.min(CSI_train_conca[i,:]), np.max(CSI_train_conca[i,:]))
print(CSI_train_conca.shape)
print(GT_train.shape)


(4680, 1500, 2)
(4680, 1)


In [45]:
# 保存前4500个样本作为训练集
CSI_train_conca = CSI_train_conca[0:4500,:]
GT_train = GT_train[0:4500,:]
print(CSI_train_conca.shape)
np.save(PathTrain + '/CSI_train_conca.npy', CSI_train_conca)
np.save(PathTrain + '/GT_train.npy', GT_train)

(4500, 1500, 2)
